In [1]:
from massspecgym.data.datasets import MSnDataset, MSnRetrievalDataset, RetrievalDataset
import torch

In [2]:
file_mgf = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_msn_library_pos_all_lib_MSn.mgf"
file_json = "/Users/macbook/CODE/Majer:MassSpecGym/data/Retrieval/MassSpecGym_retrieval_candidates_mass.json"
split_file = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/20241211_split.tsv"

In [3]:
from massspecgym.data.transforms import MolFingerprinter, SpecTokenizer
from massspecgym.data import RetrievalDataset, MassSpecDataModule

In [4]:
pth_massspecgym_original = "/Users/macbook/CODE/Majer:MassSpecGym/data/MassSpecGym/MassSpecGym.tsv"

In [5]:
# Init hyperparameters
n_peaks = 60
fp_size = 4096
batch_size = 12

# Load dataset
dataset_original = RetrievalDataset(
    pth=pth_massspecgym_original,
    candidates_pth=file_json,
    spec_transform=SpecTokenizer(n_peaks=n_peaks),
    mol_transform=MolFingerprinter(fp_size=fp_size),
)


In [6]:
# Init data module
data_module_original = MassSpecDataModule(
    dataset=dataset_original,
    batch_size=batch_size,
    num_workers=0,
)

In [7]:
data_module_original.prepare_data()
data_module_original.setup()

train_loader_original = data_module_original.train_dataloader()

Train dataset size: 194119
Val dataset size: 19429


In [8]:
tmp_original = []
for batch in train_loader_original:
    print(batch)
    tmp_original = batch
    break

I am not here
{'spec': tensor([[[3.8305e+02, 1.1000e+00],
         [6.9070e+01, 6.6066e-02],
         [7.1086e+01, 1.8018e-02],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[2.7515e+02, 1.1000e+00],
         [8.1045e+01, 4.6446e-01],
         [1.0604e+02, 2.4024e-02],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[3.6323e+02, 1.1000e+00],
         [8.6096e+01, 1.3846e-01],
         [9.1054e+01, 1.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        ...,

        [[4.0716e+02, 1.1000e+00],
         [5.5054e+01, 1.7365e-01],
         [5.5057e+01, 2.9800e-03],
         ...,
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00]],

        [[2.0715e+02, 1.1000e+00],
         [1.1505e+02, 4.

In [9]:
# Check the keys in the batch
print(tmp_original.keys())

# Specifically check if 'batch_ptr' is present
if 'batch_ptr' in tmp_original:
    print("batch_ptr is present:", tmp_original['batch_ptr'])
else:
    print("batch_ptr is missing")

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])
batch_ptr is present: tensor([256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256])


In [10]:
tmp_original.keys()

dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [11]:
tmp_original['spec'].shape, tmp_original['mol'].shape, 

(torch.Size([12, 61, 2]), torch.Size([12, 4096]))

In [12]:
tmp_original['precursor_mz']

tensor([383.0497, 275.1509, 363.2278, 150.1278, 810.6010, 141.0771, 262.1440,
        254.0594, 351.1815, 407.1602, 207.1492, 342.1547])

In [13]:
tmp_original['adduct'], 

(['[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+',
  '[M+H]+'],)

In [14]:
tmp_original['mol_freq']

tensor([200.,   8.,   3., 210.,  57.,  41.,   2., 294.,   3.,   3.,  67.,  63.])

In [15]:
tmp_original['labels']

tensor([ True, False, False,  ..., False, False, False])

In [16]:
sum(tmp_original['labels'])

tensor(12)

In [17]:
torch.nonzero(tmp_original['labels'], as_tuple=False).squeeze()

tensor([   0,  256,  512,  768, 1024, 1280, 1536, 1792, 2048, 2304, 2560, 2816])

In [18]:
len(tmp_original['labels'])

3072

In [19]:
tmp_original['batch_ptr']

tensor([256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256, 256])

In [20]:
sum(tmp_original['batch_ptr'])

tensor(3072)

In [21]:
len(tmp_original['candidates'])

3072

In [22]:
tmp_original['candidates'][5].shape

torch.Size([4096])

In [23]:
len(tmp_original['candidates_smiles'])

3072

# MSnRetrieval

In [24]:
from massspecgym.featurize import SpectrumFeaturizer

In [25]:
config = {
    'features': ['collision_energy', 'ionmode', 'adduct', 'spectrum_stats', 'atom_counts', 'value', "retention_time", 'ion_source', 'binned_peaks'],
    'feature_attributes': {
        'atom_counts': {
            'top_n_atoms': 12,
            'include_other': True,
        },
    },
}

In [26]:
featurizer = SpectrumFeaturizer(config, mode='torch')

In [27]:
file_json = "/Users/macbook/CODE/Majer:MassSpecGym/data/candidates_generation/MassSpecGymMSn_retrieval_candidates_formula.json"

In [28]:
# file_mgf = "/Users/macbook/CODE/Majer:MassSpecGym/data/MSn/min_sample_trees.mgf"


In [29]:
# Init hyperparameters
n_peaks = 60
fp_size = 4096
batch_size = 12

# Load dataset
dataset_msn = MSnRetrievalDataset(
    pth=file_mgf,
    candidates_pth=file_json,
    featurizer=featurizer,
    mol_transform=MolFingerprinter(fp_size=fp_size),
    max_allowed_deviation=0.005
)


Total valid indices: 16476
MSnRetrievalDataset length: 16476
Loading precomputed data from /Users/macbook/CODE/Majer:MassSpecGym/data/candidates_generation/msnretrieval_precomputed.pkl


EOFError: Ran out of input

In [ ]:
# Init data module
data_module_msn = MassSpecDataModule(
    dataset=dataset_msn,
    batch_size=batch_size,
    split_pth=split_file,
    num_workers=0,
)

In [ ]:
data_module_msn.prepare_data()
data_module_msn.setup()

train_loader_msn = data_module_msn.train_dataloader()

In [ ]:
tmp_msn = []
for batch in train_loader_msn:
    print(batch)
    tmp_msn = batch
    break

In [ ]:
# Check the keys in the batch
print(tmp_msn.keys())

# Specifically check if 'batch_ptr' is present
if 'batch_ptr' in tmp_msn:
    print("batch_ptr is present:", tmp_msn['batch_ptr'])
else:
    print("batch_ptr is missing")

In [ ]:
def print_key_types(obj, obj_name):
    print(f"Data types in {obj_name}:")
    for key, value in obj.items():
        print(f"  {key}: {type(value)}")

# Print data types for both objects
print_key_types(tmp_original, "tmp_original")
print_key_types(tmp_msn, "tmp_msn")

#### Shold be dict_keys(['spec', 'mol', 'precursor_mz', 'adduct', 'mol_freq', 'identifier', 'smiles', 'candidates', 'labels', 'batch_ptr', 'candidates_smiles'])

In [ ]:
tmp_msn.keys()

In [ ]:
tmp_msn['spec'] 

In [ ]:
tmp_msn['precursor_mz']

In [ ]:
tmp_msn['mol_freq']

In [ ]:
tmp_msn['identifier']

In [ ]:
tmp_msn['adduct']

In [ ]:
tmp_msn['mol'].shape

In [ ]:
tmp_msn['smiles']

In [ ]:
tmp_msn['labels']

In [ ]:
sum(tmp_msn['labels'])

In [ ]:
torch.nonzero(tmp_msn['labels'], as_tuple=False).squeeze()

In [ ]:
len(tmp_msn['labels'])

In [ ]:
tmp_msn['batch_ptr']

In [ ]:
sum(tmp_msn['batch_ptr'])

In [ ]:
len(tmp_msn['candidates'])

In [ ]:
tmp_msn['candidates'][5].shape

### Compare Shapes

In [51]:
tmp_msn.keys() == tmp_original.keys()

True

In [52]:
tmp_msn['batch_ptr'].shape == tmp_original['batch_ptr'].shape

True

In [53]:
tmp_msn['batch_ptr'].dtype == tmp_original['batch_ptr'].dtype

False

In [54]:
tmp_original['batch_ptr'].dtype

torch.int64

In [55]:
tmp_msn['batch_ptr'].dtype

torch.int32

In [56]:
sum(tmp_msn['batch_ptr']), len(tmp_msn['candidates_smiles'])

(tensor(2741, dtype=torch.int32), 2741)

In [57]:
sum(tmp_original['batch_ptr']), len(tmp_original['candidates_smiles'])

(tensor(3055), 3055)

In [58]:
tmp_msn['labels'].dtype == tmp_original['labels'].dtype

True

In [59]:
tmp_msn['candidates'].dtype == tmp_original['candidates'].dtype

True

In [60]:
tmp_msn['candidates'][0].shape == tmp_original['candidates'][0].shape

True

In [61]:
tmp_msn['candidates'][5].shape == tmp_original['candidates'][5].shape

True

In [62]:
type(tmp_msn['smiles']) == type(tmp_original['smiles'])

True

In [63]:
type(tmp_msn['smiles'][0]) == type(tmp_original['smiles'][0])

True

In [64]:
len(tmp_msn['smiles']) == len(tmp_original['smiles'])

True

In [65]:
type(tmp_msn['candidates_smiles']) == type(tmp_original['candidates_smiles'])

True

In [66]:
type(tmp_msn['candidates_smiles'][0]) == type(tmp_original['candidates_smiles'][0])

True

In [67]:
len(tmp_msn['candidates_smiles'])

2741